In [1]:
!pip install --upgrade pip
!pip install diffusers transformers accelerate pillow torch torchvision torchaudio gradio -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
import diffusers
print("✅ Diffusers installed successfully! Version:", diffusers.__version__)


✅ Diffusers installed successfully! Version: 0.35.2


In [7]:
import torch
print(torch.cuda.is_available())  # True = GPU, False = CPU


True


In [2]:
MODELS = {
    "Realistic": "runwayml/stable-diffusion-v1-5",
    "Cartoon": "prompthero/openjourney",
    "Oil Painting": "nitrosocke/mo-di-diffusion"
}

def load_model(style):
    model_id = MODELS[style]
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
    if torch.cuda.is_available():
        pipe = pipe.to("cuda")
    return pipe

pipe_cache = {}
def get_pipe(style):
    if style not in pipe_cache:
        pipe_cache[style] = load_model(style)
    return pipe_cache[style]


In [3]:
def generate(prompt, style, num_images):
    pipe = get_pipe(style)
    images = pipe([prompt] * num_images).images
    output_paths = []

    os.makedirs("outputs", exist_ok=True)
    for i, img in enumerate(images):
        path = f"outputs/{style}_{i}.png"
        img.save(path)
        output_paths.append(path)
    return output_paths


In [6]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("<h1 style='text-align:center;'>🎨 Text-to-Image Generator using Stable Diffusion</h1>")
    gr.Markdown("Turn any text prompt into an image in seconds! Choose a **style**, enter your idea, and click **Generate**.")

    with gr.Row():
        prompt = gr.Textbox(label="Enter your prompt", placeholder="e.g. a futuristic city at sunset")
        style = gr.Dropdown(choices=list(MODELS.keys()), value="Realistic", label="Select style")
        num_images = gr.Slider(1, 3, value=1, step=1, label="Number of images")

    generate_btn = gr.Button("🚀 Generate Images")
    gallery = gr.Gallery(label="Generated Images")

    gr.Examples(
        examples=[
            ["A cat wearing sunglasses on the beach", "Cartoon", 1],
            ["A portrait of an astronaut painted in oil", "Oil Painting", 1],
            ["A futuristic car racing through neon city", "Realistic", 1],
        ],
        inputs=[prompt, style, num_images]
    )

    generate_btn.click(generate, [prompt, style, num_images], gallery)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e5e09bee010166c3a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
